In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
# %config InlineBackend.print_figure_kwargs={'bbox_inches':None}
import cv2
import numpy as np
import os.path as osp
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib as mpl

mpl.rcParams['figure.dpi']= 300

def show(im, **kwargs):
    '''Show images actual size unless it is tiny
    
    I'm assuming they are in LAB float32 if the rank is 3
    
    '''
    height, width = im.shape[:2]
    if height > 50 and width > 50:
        dpi = 100
        margin= 50
        figsize=((width+2*margin)/dpi, (height+2*margin)/dpi) # inches
        left = margin/dpi/figsize[0] #axes ratio
        bottom = margin/dpi/figsize[1]

        fig = plt.figure(figsize=figsize, dpi=dpi)
        fig.subplots_adjust(left=left, bottom=bottom, right=1.-left, top=1.-bottom)
    else:
        plt.figure()
    
    args = dict(kwargs)
    if 'title' in args:
        del args['title']
    
    if len(im.shape) == 3:
        im = cv2.cvtColor(im, cv2.COLOR_LAB2RGB)
    elif len(im.shape) == 2:
        args['cmap'] = 'gray'                  

    plt.imshow(im, **args)
    if 'title' in kwargs:
        plt.title(kwargs['title'])
        
def isBlue(im):
    mblue = np.array([ 60.4 , -12.2, -35.7 ], dtype=np.float32)
    sblue = np.array([ 4.1, 3.2, 8.5], dtype=np.float32)
    d2 = np.sum((im - mblue)**2 / sblue**2, axis=2)
    return np.exp(-d2 / 20)

def circularity(contour):
    perimeter = cv2.arcLength(contour, True)
    if perimeter == 0:
        return False
    contour = cv2.convexHull(contour)
    area = cv2.contourArea(contour)
    result = 4 * np.pi * (area / perimeter ** 2)
    return result

def isCircular(contour, hull=False):
    perimeter = cv2.arcLength(contour, True)
    if perimeter == 0:
        return False
    if hull:
        contour = cv2.convexHull(contour)
    area = cv2.contourArea(contour)
    circularity = 4 * np.pi * (area / perimeter ** 2)
    return 0.7 <= circularity <= 1.2
    # return 0.5 <= circularity <= 1.4

In [ ]:
import pickle
circles = pickle.load(open('MSB_Video_1.circles', 'rb'))
print(len(circles))
print(circles[0])

In [ ]:
def rms(v):
    return np.sqrt(np.mean(v**2))

compressed = []
for f1, y1, x1, r1, v1, p1 in circles:
    for f2, y2, x2, r2, v2, p2 in compressed:
        if y1 == y2 and x1 == x2 and r1 == r2 and p1.shape == p2.shape and rms(p1 - p2) < 10:
            break
    else:
        compressed.append((f1, y1, x1, r1, v1, p1))
print(len(circles), len(compressed))

In [ ]:
def plotCircles(circles):
    circles = sorted(circles, key=lambda t: t[3])
    ncols = 25
    nrows = (len(circles) + ncols - 1) // ncols
    plt.figure(figsize=(ncols/2,nrows/2), dpi=100)
    for i, (fno, y, x, r, v, pixels) in enumerate(circles):
        plt.subplot(nrows, ncols, i+1)
        im = cv2.cvtColor(pixels, cv2.COLOR_LAB2RGB)
        plt.imshow(im)
        plt.axis('off')

In [ ]:
plotCircles(compressed)